# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.core.dataset import Dataset

from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

from azureml.core.environment import Environment 
from azureml.core.model import InferenceConfig 
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.model import Model
from azure.ai.ml import MLClient
import azure.ai.ml as aml

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)
print("azure-ai-ml versoins:", aml.__version__)

SDK version: 1.61.0
azure-ai-ml versoins: 1.31.0


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

# choose a name for experiment
experiment_name = 'odl_user_295398'
project_folder = './'

quick-starts-ws-295398
aml-quickstarts-295398
westeurope
3d1a56d2-7c81-4118-9790-f85d1acf0c77


In [3]:
from azure.identity import DefaultAzureCredential

experiment=Experiment(ws, experiment_name)
experiment

subscription_id="3d1a56d2-7c81-4118-9790-f85d1acf0c77"
resource_group="aml-quickstarts-295398"
workspace_name="quick-starts-ws-295398"

ml_client = MLClient(
    DefaultAzureCredential(),
    subscription_id=subscription_id,
    resource_group_name=resource_group,
    workspace_name=workspace_name
)

print("Connected to: ", ml_client.workspace_name)

Connected to:  quick-starts-ws-295398


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [4]:
data_asset = ml_client.data.get(name="hf-dataset", version="1")

print("name", data_asset.name)
print("version", data_asset.version)
print("type", data_asset.type)
print("path", data_asset.path)

name hf-dataset
version 1
type mltable
path azureml://subscriptions/3d1a56d2-7c81-4118-9790-f85d1acf0c77/resourcegroups/aml-quickstarts-295398/workspaces/quick-starts-ws-295398/datastores/workspaceblobstore/paths/UI/2026-02-04_105703_UTC/heart_failure_clinical_records_dataset.csv/


In [5]:
# Choose a name for your CPU cluster
cluster_name = "compute00"

compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    
if compute_target:
    print('Found existing cluster, use it.')
else: 
    print("Not found under name " + cluster_name)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [6]:
from azure.ai.ml.automl import classification
from azure.ai.ml import Input
from azure.ai.ml.constants import AssetTypes

job = classification(
    compute="compute00",
    training_data=Input(type=AssetTypes.MLTABLE, path="azureml:hf-dataset:1"),
    target_column_name="DEATH_EVENT",
    primary_metric="accuracy"
)

job.set_limits(timeout_minutes=30)
job.set_limits(max_trials=20)
job.set_limits(max_concurrent_trials=5)

created = ml_client.jobs.create_or_update(job)
print("Job name: ", created.name)

ml_client.jobs.stream(created.name)
parent_job_id = created.name


Job name:  lemon_music_vldmcfdwdk
RunId: lemon_music_vldmcfdwdk
Web View: https://ml.azure.com/runs/lemon_music_vldmcfdwdk?wsid=/subscriptions/3d1a56d2-7c81-4118-9790-f85d1acf0c77/resourcegroups/aml-quickstarts-295398/workspaces/quick-starts-ws-295398

Execution Summary
RunId: lemon_music_vldmcfdwdk
Web View: https://ml.azure.com/runs/lemon_music_vldmcfdwdk?wsid=/subscriptions/3d1a56d2-7c81-4118-9790-f85d1acf0c77/resourcegroups/aml-quickstarts-295398/workspaces/quick-starts-ws-295398



/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/mlflow/__init__.py:41: UserWarning: Versions of mlflow (3.8.1) and child packages mlflow-skinny (3.5.0) are different. This may lead to unexpected behavior. Please install the same version of all MLflow packages.
  mlflow.mismatch._check_version_mismatch()


In [11]:
from azure.ai.ml.entities import Model

# register model
parent_job = ml_client.jobs.get(parent_job_id)
#print("Parent job NAME ", parent_job)
print("Parent job ID", parent_job_id, "is ", parent_job.status)

model_name = "icy_lizard_3q0jg0t9"

run_model = Model(
    path=f"runs:/{parent_job_id}/{model_name}",
    name="best-heart-failure-model",     
    description="Model created from run.",
    type=AssetTypes.MLFLOW_MODEL
)
model_asset = ml_client.models.create_or_update(run_model)
print(f"Created Model asset: {model_name}")


Parent job ID lemon_music_vldmcfdwdk is  Completed


HttpResponseError: (UserError) The request is invalid.
Code: UserError
Message: The request is invalid.
Exception Details:	(NoMatchingArtifactsFoundFromRun) No artifacts matching icy_lizard_3q0jg0t9 found from Run.
	Code: NoMatchingArtifactsFoundFromRun
	Message: No artifacts matching icy_lizard_3q0jg0t9 found from Run.
Additional Information:Type: ComponentName
Info: {
    "value": "managementfrontend"
}Type: Correlation
Info: {
    "value": {
        "operation": "3f90e8a6f88bfad3d5722b2188b75729",
        "request": "e6a9d15bb5b4761d"
    }
}Type: Environment
Info: {
    "value": "westeurope"
}Type: Location
Info: {
    "value": "westeurope"
}Type: Time
Info: {
    "value": "2026-02-04T11:59:26.0111402+00:00"
}

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [13]:
import uuid

# Creating a unique name for the endpoint
online_endpoint_name = "hf-endpoint-" + str(uuid.uuid4())[:8]

In [14]:
from azure.ai.ml.entities import ManagedOnlineEndpoint

# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description="online endpoint with key auth",
    auth_mode="key",
    tags={
        "training_dataset": "heart-failure-dataset",
        "model_type": "automl.job = classification",
    },
)

endpoint = ml_client.online_endpoints.begin_create_or_update(endpoint).result()

print(f"Endpoint {endpoint.name} provisioning state: {endpoint.provisioning_state}")

Endpoint hf-endpoint-af2f866d provisioning state: Succeeded


In [15]:
endpoint = ml_client.online_endpoints.get(name=online_endpoint_name)

print(
    f'Endpoint "{endpoint.name}" with provisioning state "{endpoint.provisioning_state}" is retrieved'
)

Endpoint "hf-endpoint-af2f866d" with provisioning state "Succeeded" is retrieved


## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
